# Two-Tier Urban Deliveries with Robots with time windows

In [1]:
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter  # for some testing [optional]
import ipywidgets as widgets  # for interactive plots [optional]
from ipywidgets import interact, interact_manual  # for interactive plots [optional]
from tqdm.notebook import tqdm  # for progress bars [optional]

# program must be adjusted in some places if optional packages are not available

#### Some custom functions for later use

In [2]:
def grid (a, n=None, spacing=None):
    """
    distribute n points evenly in a 2-dimensional square of side length a
    or create a square grid with suqare cells of length c
    """
    if spacing is None:
        assert n
        spacing = a // int(np.sqrt(n))
        base = range(spacing//2, a, spacing)
    if n is None:
        assert spacing
        base = range(0, a+1, spacing)
    return[(x,y) for x in base for y in base]

def travel_time(first: tuple, second:tuple, speed):
    """
    compute the travel time between first and second
    first, second: given as (x, y) tuples 
    speed: given in km/h
    """
    dist = abs(first[0] - second[0]) + abs(first[1] - second[1])  # Manhattan distance in m (!) because input is in meters
    return (dist/1000)/speed  # travel time in hours



#### Model inputs
If the solver finds the model to be infeasible, first check whether there are enough capacities (robots, hubs) to serve all the customers and adjust these numbers to see whether it fixes the issue.

In [3]:
np.random.seed(0)  # for reproducible results

n = 100  # number of clients
r_max = 10  # number of robots per hub
num_instances = 10  # number of instances

# downtown map
area= 2000  # 2km * 2km square
block = 100  # 100m * 100m blocks
h = 16  # number of potential hubs (should be a square number (4, 9, 16, 25, ...) for an even distribution)

# suburban map
# area = 10000
# block = 100
# h= 36

M = 6  # maximum allowed driving time per robot (hrs)
tf = 45/60  # robots' full recharge time (hrs)
b = 2  # robots' battery range (hrs)
ts = 4/60  # customer service time (hrs)
v = 3  # robot speed (km/h)
short_tw = [(i, i+1) for i in range(8, 16)]  # 1 hour time windows
long_tw = [(i, i+2) for i in range(8, 16, 2)]  # 2 hour time windows
long_tw_demand = (0.25, 0.1, 0.1, 0.55)  # time window demand
L = 17  # end of time horizon -> NOT GIVEN IN THE PAPER

H = [i for i in range(h)]  # hubs
R = [i for i in range(r_max)]  # robots
N = [i for i in range(n)]  # customers
I = [i for i in range(num_instances)]  # instances

In [4]:
hub_coords = grid(a=area, n=h)

grid_x, grid_y = zip(*grid(a=area, spacing=block))
customer_coords = gp.tupledict()
for instance in I:  # originally, smaller instances are subsets of the bigger ones (n=300), here all instances are unique
    for customer in N:
        customer_coords[instance, customer] = (np.random.choice(grid_x), np.random.choice(grid_y))

# customer time windows
tw_keys = [(instance, customer) for instance in I for customer in N]
tw_values = [long_tw[i] for i in np.random.choice(range(len(long_tw)), len(tw_keys), p=long_tw_demand)]
time_windows = gp.tupledict({k:v for k, v in zip(tw_keys, tw_values)})  

t = gp.tupledict()  # pendulum distances
for hub in tqdm(H, desc='Pendulum distances'):
    for robot in R:
        for instance in I:
            for customer in N:
                t[hub, robot, instance, customer] = 2*travel_time(first=hub_coords[hub],
                                                                  second=customer_coords[instance, customer],
                                                                  speed=v)

reachables = gp.tuplelist([key for (key, value) in t.items() if value <=b])  # within battery range of each hub

reachable_customers = gp.tupledict()  # dictionary of customers that are in reach of each (hub, robot, instance) combination
for hub in tqdm(H, desc='Reachable customers'):
    for robot in R:
        for instance in I:
            in_reach = []
            for customer in N:    
                if t.select(hub, robot, instance, customer)[0] <= b:
                    in_reach.append(customer)
                reachable_customers[(hub, robot, instance)] = in_reach

z_vars = []
valid_precedences = gp.tupledict()
for hub in tqdm(H, desc='Valid precedence combinations'):
    for robot in R:
        for instance in I:
            valid_precedences[hub, robot, instance] =[]
            for i in reachable_customers[hub, robot, instance]:
                for j in reachable_customers[hub, robot, instance]:
                    if time_windows[instance, j] >= time_windows[instance, i] and i != j:
                        z_vars.append((hub, robot, instance, i, j))
                        valid_precedences[hub, robot, instance].append((i,j))

#### Plotting hubs and a single instance

In [5]:
colors = plt.get_cmap('hsv', h)
hubs_x, hubs_y = zip(*hub_coords)

def base_plot(plot_instance:int, which):
    plt.scatter(hubs_x, hubs_y, cmap=colors, c=H, marker='s', s=75, edgecolors='black')
    
    customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))
    plt.scatter(customers_x, customers_y, c='black', alpha=0.3)
    for customer in N:
        if which == 'customer_tw':
            plt.annotate(s=time_windows[plot_instance, customer], xy=(customers_x[customer], customers_y[customer]))
        else:
            plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))
        
    plt.xlim(0, area)
    plt.ylim(0, area)
    plt.grid()
    plt.gcf().set_size_inches(18,6)
interact(base_plot, plot_instance=I, which=['customer_id', 'customer_tw']);

interactive(children=(Dropdown(description='plot_instance', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), …

### Tier 1: Minimize the number of hubs

#### Model, decision variables and objective function

In [6]:
model1 = gp.Model('min_hubs')

# (5) add hub-robot-instance-customer binary decision variables
x1 = model1.addVars(reachables, vtype=GRB.BINARY, name='x')
# format of x1 vars: (hub, robot, instance, customer)

# (6) add is-hub-open binary decision variables
o1 = model1.addVars(H, vtype=GRB.BINARY, name='o')

# (14) add depot leaving time decision variables
y1 = model1.addVars(reachables, vtype=GRB.CONTINUOUS, name='y')
# format of y1 vars: (hub, robot, instance, customer)

# (14) add precedence decision variables
z1 = model1.addVars(z_vars, vtype=GRB.BINARY, name='z')
# format of z1 vars: (hub, robot, instance, customer i, customer j)

Using license file C:\Users\steff\gurobi.lic
Academic license - for non-commercial use only


In [7]:
# (1) set objective
model1.setObjective(gp.quicksum(o1), sense=GRB.MINIMIZE)
model1.update()

#### Adding constraints

In [8]:
# (2) All instance-customer locations must be assigned to exactly one hub-robot combination
for instance in tqdm(I, desc='Service constraints'):
    for customer in N:
        model1.addConstr(x1.sum('*', '*', instance, customer) == 1)

In [9]:
# (3) limit on the maximum robot working time
coeff1 = gp.tupledict({
    (hub, robot, instance, customer): t[hub, robot, instance, customer] * (1 + (tf / b)) + ts
    for hub in H
    for robot in R
    for instance in I
    for customer in N
})

for hub in tqdm(H, desc='Maximum working time constraints'):
    for robot in R:
        for instance in I:
            model1.addConstr(x1.prod(coeff1, hub, robot, instance, '*') <= M)

In [10]:
# (4) if a robot serves a customer location (in some instance), the corresponding robot hub is open
for hub in tqdm(H, desc='Active hubs constraints'):
    for robot in R:
        for instance in I:
            for customer in reachable_customers[hub, robot, instance]:
                model1.addConstr(x1.sum(hub, instance, customer, '*') <= o1[hub])

In [11]:
# (9), (10) time windows
for hub in tqdm(H, desc='Time window constraints'):
    for robot in R:
        for instance in I:
            for customer in reachable_customers[hub, robot, instance]:
                e = time_windows[instance, customer][0]  # time window open
                l = time_windows[instance, customer][1]  # time window close               
                coeff = t[hub, robot, instance, customer]/2  # one-way travel time
                x = x1[hub, robot, instance, customer]
                y = y1[hub, robot, instance, customer]
                
                model1.addConstr(e * x <= y + coeff * x)
                model1.addConstr(y + coeff * x <= l * x)

In [12]:
# (11) the next customer cannot be served before service at the previous one has finished and the battery is fully recharged
for hub in tqdm(H, desc='Time integrity constraints'):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                if time_windows[instance, j] >= time_windows[instance, i] and i != j:
                    y_i = y1[hub, robot, instance, i]
                    y_j = y1[hub, robot, instance, j]
                    z_ij = z1[hub, robot, instance, i, j]

                    model1.addConstr(y_j + (L  + tf / b) * (1 - z_ij) >= y_i + t[hub, robot, instance, i] * (1 + (tf / b)) + ts)

In [13]:
# (12) connecting x and z variables I
for hub in tqdm(H, desc=''):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                if time_windows[instance, j] == time_windows[instance, i] and i != j:  # where time windows are same
                    x_i = x1[hub, robot, instance, i]
                    x_j = x1[hub, robot, instance, j]
                    z_ij = z1[hub, robot, instance, i, j]
                    z_ji = z1[hub, robot, instance, j, i]

                    model1.addConstr(x_i + x_j - z_ij - z_ji <= 1)

In [14]:
# (13) connecting x and z variables II
for hub in tqdm(H):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                # if time_windows[instance, j] >= time_windows[instance, i] and i != j:  # where tw of j is equal or later i
                # illogical: if tw[j] > tw[i] then there is no variable z_ji
                if time_windows[instance, j] == time_windows[instance, i] and i != j:  # where tw are equal
                    x_i = x1[hub, robot, instance, i]
                    x_j = x1[hub, robot, instance, j]
                    z_ji = z1[hub, robot, instance, j, i]

                    model1.addConstr(x_i + x_j - z_ji <= 1)

#### Solve Model 1

In [15]:
model1.setParam('TimeLimit', 240)  # for testing only
model1.optimize()

Changed value of parameter TimeLimit to 240.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 23633830 rows, 11173786 columns and 78297430 nonzeros
Model fingerprint: 0xfe0cc25f
Variable types: 158580 continuous, 11015206 integer (11015206 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 0 columns (presolve time = 8s) ...
Presolve removed 142730 rows and 4707170 columns (presolve time = 12s) ...
Presolve removed 4849900 rows and 4707170 columns (presolve time = 16s) ...
Presolve removed 7938885 rows and 4707170 columns (presolve time = 39s) ...
Presolve removed 7938885 rows and 4707170 columns (presolve time = 49s) ...
Presolve removed 7938885 rows and 4707170 columns (presolve time = 50s) ...
Presolve removed 17085637 rows and 4707170 columns (presolve time = 55s) .

#### Inspect and plot one of the instances with the solution

In [16]:
x1_solution = model1.getAttr('x', x1)
assignment1 = gp.tupledict({
    (key[2], key[3]):(key[0], key[1]) 
    for key, value in x1_solution.items() 
    if value > 0.5})  # (instance, customer): (hub, robot)
assignment1;

In [17]:
o1_solution = model1.getAttr('x', o1)
p = len([key for key, value in o1_solution.items() if value > 0.5])  # min number of open hubs
p

10

In [18]:
def model1_plot(plot_instance):
    hub_colors = []
    for hub in H:
        if o1_solution[hub] > 0.5:
            hub_colors.append(colors(hub/h))
        else:
            hub_colors.append('white')

    plt.scatter(hubs_x, hubs_y, c=hub_colors, marker='s', s=75, edgecolors='black')
    
    if plot_instance is not None:
        customer_colors = []
        for customer in N:
            customer_colors.append(colors(assignment1[plot_instance, customer][0]/h))  

        customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))  
        plt.scatter(customers_x, customers_y, c=customer_colors, alpha=1, edgecolors='black')

        if n <= 100:
            for customer in N:
                plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))

    plt.xlim(0, area)
    plt.ylim(0, area);
    plt.grid()
    # plt.gcf().set_size_inches(20,10)
interact(model1_plot, plot_instance=[None]+I);

interactive(children=(Dropdown(description='plot_instance', options=(None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9), valu…

### Tier 2: Minimize operational robot cost

#### Model, decision variables, objective function

In [19]:
model2 = gp.Model('min_cost')

# (5) add hub-robot-instance-customer binary decision variables
x2 = model2.addVars(reachables, vtype=GRB.BINARY, name='x')
# format of x2 vars: (hub, robot, instance, customer)

# (6) add is-hub-open binary decision variables
o2 = model2.addVars(H, vtype=GRB.BINARY, name='o')

# (14) add depot leaving time decision variables
y2 = model2.addVars(reachables, vtype=GRB.CONTINUOUS, name='y')
# format of y2 vars: (hub, robot, instance, customer)

# (14) add precedence decision variables
z2 = model2.addVars(z_vars, vtype=GRB.BINARY, name='z')
# format of z2 vars: (hub, robot, instance, customer i, customer j)

# set the objective function
coeff2 = gp.tupledict({
    (hub, robot, instance, customer): t[hub, robot, instance, customer]
    for hub in H
    for robot in R
    for instance in I
    for customer in N
})

model2.setObjective(x2.prod(coeff2), sense=GRB.MINIMIZE)
model2.update()
# model2.getObjective()

#### Add constraints

In [20]:
# define the constraints
# (2) All instance-customer locations must be assigned to exactly one hub-robot combination
for instance in tqdm(I, desc='Service constraints'):
    for customer in N:
        model2.addConstr(x2.sum('*', '*', instance, customer) == 1)

In [21]:
# (3) limit on the maximum robot working time
# following the paper, this constraint is NOT applied for tier 2 with tw ?!
"""
coeff1 = gp.tupledict({
    (hub, robot, instance, customer): t[hub, robot, instance, customer] * (1 + (tf / b)) + ts
    for hub in H
    for robot in R
    for instance in I
    for customer in N
})

for hub in tqdm(H, desc='Maximum working time constraints'):
    for robot in R:
        for instance in I:
            model2.addConstr(x2.prod(coeff1, hub, robot, instance, '*') <= M)
""";
     

In [22]:
# (4) if a robot serves a customer location (in some instance), the corresponding robot hub is open
for hub in tqdm(H, desc='Active hubs constraints'):
    for robot in R:
        for instance in I:
            for customer in reachable_customers[hub, robot, instance]:
                model2.addConstr(x2.sum(hub, instance, customer, '*') <= o2[hub])

In [23]:
# (8) ensure that there are exactly as many open robot hubs as provided by first model
model2.addConstr(o2.sum() == p);

In [24]:
# (9), (10) time windows
for hub in tqdm(H, desc='Time window constraints'):
    for robot in R:
        for instance in I:
            for customer in reachable_customers[hub, robot, instance]:
                e = time_windows[instance, customer][0]  # time window open
                l = time_windows[instance, customer][1]  # time window close               
                coeff = t[hub, robot, instance, customer]/2  # one-way travel time
                x = x2[hub, robot, instance, customer]
                y = y2[hub, robot, instance, customer]
                
                model2.addConstr(e * x <= y + coeff * x)   # time window open
                model2.addConstr(y + coeff * x <= l * x)   # time window close 

In [25]:
# (11) the next customer cannot be served before service at the previous one has finished and the battery is fully recharged
for hub in tqdm(H, desc='Time integrity constraints'):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                if time_windows[instance, j] >= time_windows[instance, i] and i != j:
                    y_i = y2[hub, robot, instance, i]
                    y_j = y2[hub, robot, instance, j]
                    z_ij = z2[hub, robot, instance, i, j]

                    model2.addConstr(y_j + (L  + tf / b) * (1 - z_ij) >= y_i + t[hub, robot, instance, i] * (1 + (tf / b)) + ts)

In [26]:
# (12) connecting x and z variables I
for hub in tqdm(H, desc=''):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                if time_windows[instance, j] == time_windows[instance, i] and i != j:  # where time windows are same
                    x_i = x2[hub, robot, instance, i]
                    x_j = x2[hub, robot, instance, j]
                    z_ij = z2[hub, robot, instance, i, j]
                    z_ji = z2[hub, robot, instance, j, i]

                    model2.addConstr(x_i + x_j - z_ij - z_ji <= 1)

In [27]:
# (13) connecting x and z variables II
for hub in tqdm(H):
    for robot in R:
        for instance in I:
            for i, j in valid_precedences[hub, robot, instance]:
                # if time_windows[instance, j] >= time_windows[instance, i] and i != j:  # where tw of j is equal or later i
                # illogical: if tw[j] > tw[i] then there is no variable z_ji
                if time_windows[instance, j] == time_windows[instance, i] and i != j:  # where tw are equal
                    x_i = x2[hub, robot, instance, i]
                    x_j = x2[hub, robot, instance, j]
                    z_ji = z2[hub, robot, instance, j, i]

                    model2.addConstr(x_i + x_j - z_ji <= 1)

#### Solve Model 2

In [28]:
model2.setParam('TimeLimit', 240)  # for testing only
model2.optimize()

Changed value of parameter TimeLimit to 240.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 23632231 rows, 11173786 columns and 78138866 nonzeros
Model fingerprint: 0xb71c519b
Variable types: 158580 continuous, 11015206 integer (11015206 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [7e-02, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 897.8666667
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve removed 142730 rows and 4707170 columns (presolve time = 11s) ...
Presolve removed 4849900 rows and 4707170 columns (presolve time = 16s) ...
Presolve removed 7938885 rows and 4707170 columns (presolve time = 41s) ...
Presolve removed 7938885 rows and 4707170 columns (presolve time = 51s) ...
Presolve removed 17084052 rows and 4707170 columns (presolve time = 55s) ...
Presolve removed 22783720

#### Inspect and plot solution of Model 2

In [29]:
objValue2 = model2.getObjective().getValue()
objValue2

248.66666666666697

In [30]:
x2_solution = model2.getAttr('x', x2)
# customer: (hub, robot) lookup dict
assignment2 = gp.tupledict({
    (key[2], key[3]):(key[0], key[1]) 
    for key, value in x2_solution.items() 
    if value > 0.5})  # (instance, customer): (hub, robot)
assignment2;
# test whether the constraints are satisfied: every customer is assigned to one robot and one hub
Counter([x2_solution.sum('*', '*', instance, customer).getValue() == 1 for customer in N for instance in I])

Counter({True: 1000})

In [31]:
# test the constraints: maximum working time of robots
Counter([x2.prod(coeff1, hub, robot, instance, '*').getValue() <= M for hub in H for instance in I for robot in R])

Counter({True: 1600})

In [32]:
o2_solution = model2.getAttr('x', o2)
o2_solution
# test whether constraints are satisfied: only p hubs are open
o2_solution.sum().getValue() == p

True

In [33]:
workload = dict()
for hub in H:
    for robot in R:
        for instance in I:
            wl = x2_solution.sum(hub, robot, instance, '*').getValue()
            if wl > 0:
                customers = [key[1] for key, value in assignment2.items() if value == (hub, robot) and key[0] == instance]
                workload[hub, robot, instance] = [customers, 
                                                  wl,
                                                  x2_solution.prod(coeff2, hub, robot, instance, '*').getValue(),  # pure travel time over all instances
                                                  x2_solution.prod(coeff1, hub, robot, instance, '*').getValue()  # total time incl. recharge etc.
                                       ]
active_robots, wl_customers, wl_num_customers, wl_travel_time, wl_total_time = gp.multidict(workload)
active_robots, wl_customers, wl_num_customers, wl_travel_time, wl_total_time;
wl_customers

{(0, 0, 8): [47, 56, 70, 86],
 (0, 1, 8): [31, 36, 65],
 (0, 2, 5): [27, 41, 95],
 (0, 6, 5): [86],
 (0, 8, 5): [35],
 (0, 8, 8): [34, 85],
 (1, 0, 0): [80],
 (1, 0, 1): [31],
 (1, 0, 2): [37],
 (1, 0, 3): [36],
 (1, 0, 4): [83],
 (1, 0, 5): [46],
 (1, 0, 6): [45],
 (1, 0, 7): [13],
 (1, 0, 8): [26],
 (1, 0, 9): [10],
 (1, 1, 0): [47],
 (1, 1, 1): [2],
 (1, 1, 2): [23],
 (1, 1, 3): [96],
 (1, 1, 4): [79],
 (1, 1, 5): [99],
 (1, 1, 6): [76],
 (1, 1, 7): [73],
 (1, 1, 9): [11],
 (1, 2, 0): [7],
 (1, 2, 1): [73],
 (1, 2, 2): [65],
 (1, 2, 3): [24],
 (1, 2, 4): [13],
 (1, 2, 5): [98],
 (1, 2, 6): [73],
 (1, 2, 7): [18],
 (1, 2, 8): [77],
 (1, 2, 9): [28],
 (1, 3, 0): [44],
 (1, 3, 1): [36],
 (1, 3, 2): [86],
 (1, 3, 3): [79],
 (1, 3, 4): [27],
 (1, 3, 5): [17],
 (1, 3, 6): [0],
 (1, 3, 7): [42],
 (1, 3, 8): [55],
 (1, 3, 9): [19],
 (1, 4, 0): [10],
 (1, 4, 1): [21],
 (1, 4, 2): [98],
 (1, 4, 3): [0],
 (1, 4, 4): [58],
 (1, 4, 5): [18],
 (1, 4, 6): [91],
 (1, 4, 7): [62],
 (1, 4, 9): [34],


In [34]:
def model2_plot(plot_instance):
    # plot hubs
    hub_colors = []
    for hub in H:
        if o2_solution[hub] > 0.5:
            hub_colors.append(colors(hub/h))
        else:
            hub_colors.append('white')
    plt.scatter(hubs_x, hubs_y, c=hub_colors, marker='s', s=75, edgecolors='black')
    
    # plot customers
    customer_colors = []
    for customer in N:
        customer_colors.append(colors(assignment2[plot_instance, customer][0]/h))
    customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))
    plt.scatter(customers_x, customers_y, c=customer_colors, alpha=0.5, edgecolors='black')

    for customer in N:
        plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))

    # plot robot tours, different line style per robot
    for hub, robot, instance in active_robots.select('*', '*', plot_instance):
        C = wl_customers[hub, robot, instance]
        for customer in C:
            plt.plot([hubs_x[hub], customers_x[customer]],
                     [hubs_y[hub], customers_y[customer]],
                     c=hub_colors[hub],
                     zorder=0
                    )

    plt.xlim(0, area)
    plt.ylim(0, area)
    plt.gcf().set_size_inches(20,10)
    plt.grid()
    # obviously, this plot does not show the actual tours which are based on manhatten distances
interact(model2_plot, plot_instance=I);

interactive(children=(Dropdown(description='plot_instance', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), …